In [ ]:
!nvidia-smi

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp "./drive/My Drive/Study/config/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download "birdcall-spectrogram-images"
!unzip birdcall-spectrogram-images.zip > /dev/null
!rm birdcall-spectrogram-images.zip

In [ ]:
import numpy as np
import pandas as pd
import os
import tqdm
import random
import time

import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision.models import resnet18
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import CosineAnnealingLR

import matplotlib.pyplot as plt

from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

from contextlib import contextmanager
from typing import Optional
import logging

device = torch.device('cuda')

In [ ]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
    
def get_logger(out_file=None):
    logger = logging.getLogger()
    formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
    logger.handlers = []
    logger.setLevel(logging.INFO)

    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
    handler.setLevel(logging.INFO)
    logger.addHandler(handler)

    if out_file is not None:
        fh = logging.FileHandler(out_file)
        fh.setFormatter(formatter)
        fh.setLevel(logging.INFO)
        logger.addHandler(fh)
    logger.info("logger set up")
    return logger
    
    
@contextmanager
def timer(name: str, logger: Optional[logging.Logger] = None):
    t0 = time.time()
    msg = f"[{name}] start"
    if logger is None:
        print(msg)
    else:
        logger.info(msg)
    yield

    msg = f"[{name}] done in {time.time() - t0:.2f} s"
    if logger is None:
        print(msg)
    else:
        logger.info(msg)

In [ ]:
class config:
    SEED = 416
    N_FOLDS = 5
    INPUT =  "./train_jpg/"
    N_LABEL = 264
    
    TRAIN_BS = 256
    VALID_BS = 256
    TRAIN_WORKS = 0
    VALID_WORKS = 0
    
    DROPOUT_RATE = 0.2
    N_UNIT = 512

    EPOCHS = 20
    RESNET_LR = 1e-4
    FC_LR = 1e-3
    
    FOLD = 0
    PRETRAINED = True

In [ ]:
logger = get_logger("main.log")

In [ ]:
def get_dataloder():
    train_transform = transforms.Compose([
        transforms.RandomCrop((128, 313), pad_if_needed=True, padding_mode="constant"),
        transforms.ToTensor()
    ])
    valid_transform = transforms.Compose([
        transforms.CenterCrop((128, 313)),
        transforms.ToTensor()
    ])

    train_datasets = datasets.ImageFolder(root=config.INPUT, transform=train_transform)
    valid_datasets = datasets.ImageFolder(root=config.INPUT, transform=valid_transform)

    skf = StratifiedKFold(n_splits=config.N_FOLDS, shuffle=True, random_state=config.SEED)

    _t = train_datasets.targets
    trn_idx, val_idx = [(trn_idx, val_idx) for trn_idx, val_idx in skf.split(_t, _t)][config.FOLD]

    train_datasets = torch.utils.data.Subset(train_datasets, trn_idx)
    valid_datasets = torch.utils.data.Subset(valid_datasets, val_idx)

    train_data_loader = torch.utils.data.DataLoader(train_datasets, batch_size=config.TRAIN_BS, shuffle=True, num_workers=config.TRAIN_WORKS)
    valid_data_loader = torch.utils.data.DataLoader(valid_datasets, batch_size=config.VALID_BS, shuffle=False, num_workers=config.VALID_WORKS)
    
    return train_data_loader, valid_data_loader

#for d in train_data_loader:
#    break
#img = d[0][0]
#plt.imshow(np.rollaxis(img.numpy(), 0, 3))

#with timer(f"Loading bs=32", logger):
#    for d in train_data_loader:
#        break

In [ ]:
def loss_fn(output, target):
    loss = nn.BCEWithLogitsLoss()(output, target)
    return loss

class BirdcallNet(nn.Module):
    def __init__(self):
        super(BirdcallNet, self).__init__()
        
        self.resnet = resnet18(pretrained=config.PRETRAINED)
        self.resnet_head = list(self.resnet.children())
        self.resnet_head = nn.Sequential(*self.resnet_head[:-1])
        
        self.dropout = nn.Dropout(p=config.DROPOUT_RATE)
        self.fc = nn.Linear(config.N_UNIT, config.N_LABEL)

    def forward(self, x):
        h = self.resnet_head(x)
        h = self.dropout(h.view(-1, config.N_UNIT))
        logits = self.fc(h)
        return logits

In [ ]:
def train_fn(train_data_loader, model, optimizer, scheduler=None):
    losses, lrs = [], []
    model.train()
    t = tqdm.notebook.tqdm(train_data_loader, total=len(train_data_loader))
    for (X, y) in t:
    
        y_true = torch.eye(config.N_LABEL)[y].to(device)
    
        y_pred = model(X.to(device))
    
        loss = loss_fn(y_pred, y_true)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
        
        t.set_description(f"Train Loss = {loss.item()}")

        losses.append(loss.item())
        lrs.append(np.array([param_group["lr"] for param_group in optimizer.param_groups]).mean())
    return losses, lrs

        
def valid_fn(valid_data_loader, model):
    f1_lst = []
    model.eval()
    t = tqdm.notebook.tqdm(valid_data_loader, total=len(valid_data_loader))
    for (X, y) in t:

        with torch.no_grad():
            y_pred = model(X.to(device))

        f1 = f1_score(y, y_pred.sigmoid().argmax(1).cpu(), average="micro")
        f1_lst.append(f1)
    return sum(f1_lst)/len(f1_lst)

In [ ]:
print(f"### Fold-{config.FOLD} ###")

set_seed(config.SEED+config.FOLD)

train_data_loader, valid_data_loader = get_dataloder()

model = BirdcallNet()
#weights_path = "../input/birdcall-trained-models/birdcallnet_f0_best.bin"
#model.load_state_dict(torch.load(weights_path))
model.to(device)

optimizer = Adam([{'params': model.resnet.parameters(), 'lr': config.RESNET_LR},
                  {'params': model.fc.parameters(), 'lr': config.FC_LR}])
scheduler = CosineAnnealingLR(optimizer, T_max=20, eta_min=0.0)

best_score = 0
trn_losses, trn_lrs, val_scores = [], [], []
for epoch in range(config.EPOCHS):
    print(f"{epoch} epoch")
    losses, lrs = train_fn(train_data_loader, model, optimizer, scheduler)
    val_f1 = valid_fn(valid_data_loader, model)
    val_scores.append(val_f1)
    trn_losses.extend(losses)
    trn_lrs.extend(lrs)
    if best_score <= val_f1:
        best_score = val_f1
        torch.save(model.state_dict(), f"birdcallnet_f{config.FOLD}_best.bin")
        print(f"Best Score Update!!! -> {best_score}")

In [ ]:
plt.plot(trn_lrs)

In [ ]:
plt.plot(trn_losses)

In [ ]:
plt.plot(val_scores)

In [ ]:
max(val_scores)